In [289]:
# If using a conda environment, install spacy this way
#! conda install spacy

# if using .venv, install spacy this way
#! pip install spacy

# Run this to download the small English language model for spacy
#python -m spacy download en_core_web_sm

In [2]:
# Import the dependencies
import re 
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
# Set the maximum column width to 200. 
pd.set_option('max_colwidth', 200)

# Import spacy library
import spacy
# Load the small English language model for spacy
nlp = spacy.load("en_core_web_sm")

In [3]:
# Read the bbc_news_articles.csv file into a DataFrame.
d4_emails_df = pd.read_csv('../resources/scrubbed_data.csv')
# Display the first 5 rows. 
d4_emails_df.head()

,name,affected_address,email_address,case_number,date,constituent_email_1,d4_response_1,d4_staff_member,constituent_email_2,d4_response_2
0,Ron,6864 East Bucknell Place,ron@email.com,0,2024-08-05,The lack of police presence and code enforcement is sending a growing message that these violations are not important…and that reckless behavior is not of great concern. Second item: affordable de...,"Good morning Ron, \n\nThank you for reaching out, and I apologize for the delayed response. Council Pro Tem Romero Campbell maintains regular communication with DPD District 3, which serves Southe...",Staff1,NaN,NaN
1,Renee,Dahlia & High Line intersection,renee@email.com,0,2024-08-19,I would like to know if there is a possible to put a yellow flashing pedestrian light on Dahlia and the intersection of the Highline Canal.. Also is there aywya that a turn signal could be impleme...,Thank you for reaching out to our office. The area engineers have looked at this intersection where the High Line Canal crosses Dahlia multiple times at our request and have determined that a fla...,Staff2,NaN,NaN
2,JW,Happy Canyon & Jasmine St,jw@email.com,0,2024-08-19,"Accident took place at Happy Canyon & Jasmine. Last week an extremely careless driver drove at a high rate of speed and got stuck on the median island. Took out a tree, dug big holes in the ground...","Good morning TJ,\n\nThank you for reaching out to our office and for bringing this to our attention. I have reported this to the city via 311 and it will be directed to the parks and recreation te...",Staff1,NaN,NaN
3,Clara,Hutchinson Hills,clara@email.com,9578014,2024-08-20,"Has many concerns about getting a compost cart, had many issues with Solid Waste and the person they were talking to. Been trying to get a compost cart for 2 years since moving into their house. N...","Thank you for reaching out to our office and for your thoughtful email. I hope I can touch on all your points in my response.\nUnfortunately, it is not possible for you to pick up a compost bin. ...",Staff2,"Clara sent another email back with more questions about how to volunteer, wants a new rec center, and still wants more info on compost cart.",Did not respond as they were the same questions I had answewred previously
4,Pauline,NaN,pauline@email.com,0,2024-08-21,"At any rate my concern is how will HOA condo units be charges? $150.00 per unit per year so a 200 unit building would pay $40,000 additional on storm water bill. I look forward to clarification o...","Good afternoon Pauline, \n\nThank you for reaching out to our office. As you know the sidewalk fee in November 2022, a citizen-led ballot initiative passed by voters directed Denver's Department o...",Staff1,NaN,NaN


## Preprocessing

In [292]:
# Check for null values.
d4_emails_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   name                 57 non-null     object
 1   affected_address     40 non-null     object
 2   email_address        57 non-null     object
 3   case_number          57 non-null     int64 
 4   date                 57 non-null     object
 5   constituent_email_1  57 non-null     object
 6   d4_response_1        57 non-null     object
 7   d4_staff_member      57 non-null     object
 8   constituent_email_2  11 non-null     object
 9   d4_response_2        12 non-null     object
dtypes: int64(1), object(9)
memory usage: 4.6+ KB


In [293]:
# Use Parts of Speech (POS) tagging to extract the nouns, adjectives, and verbs from the constituent_email_1 column.

# Create a function that takes a string and returns only the nouns and verbs.
def get_nouns_adjs_verbs(text: str) -> str:
    tokens = nlp(text)
    nouns_adjs_verbs = ' '.join([token.text for token in tokens if token.pos_ == "NOUN" or token.pos_ == "ADJ" or token.pos_ == "VERB"])
    unique = set(nouns_adjs_verbs.lower().split())
    return ' '.join(unique)
    
d4_emails_df['nouns_adjs_verbs'] = d4_emails_df['constituent_email_1'].apply(get_nouns_adjs_verbs)
d4_emails_df['nouns_adjs_verbs']    

0          sending concern affordable second item police great enforcement lack tax message wanting more denver violations behavior set important accomplish code goals presence reckless growing information
1                                                                                                       put pedestrian light is aywya signal implemented intersection turn like flashing yellow possible know
2     place got removal speed road report various spilled driver fluids person stuff big stuck escape week holes island is dug city truck took inform drove oversaw last imagine high ground trying fixes ...
3                                                                                                needs get compost many issues person house years has talking concerns message cart moving had getting trying
4     have general be unit units wage concern year budgets clarification inflation tax rules condo losing building lookup possible class made mentioned end scoop new middle reg

In [294]:
topic_dict = {
    'homeless': 'Homeless',
    'shelter': 'Homeless',
    'encampment': 'Homeless',
    'graffiti': 'Graffiti',
    'paint': 'Graffiti',
    'pothole': 'Pothole',
    'holes': 'Pothole',
    'animal': 'Animal',
    'pest': 'Animal',
    'weeds': 'Vegitation',
    'trees': 'Vegitation',
    'overgrown': 'Vegitation',
    'neighborhood': 'Neighborhood',
    'hoa': 'Neighborhood',
    'rno': 'Neighborhood',
    'sidewalk': 'Neighborhood',
    'sidewalks': 'Neighborhood',
    'fence': 'Neighborhood',
    'fences': 'Neighborhood',
    'snow': 'Snow Removal',
    'ice': 'Snow Removal',
    'plows': 'Snow Removal',
    'vehicle': 'Vehicle',
    'vehicles': 'Vehicle',
    'car': 'Vehicle',
    'cars': 'Vehicle',
    'motorcycle': 'Vehicle',
    'motorcycles': 'Vehicle',
    'parking': 'Parking',
    'police': 'Police',
    'gang': 'Police',
    'loud': 'Police',
    'drugs': 'Police',
    'crime': 'Police',
    'firework': 'Fireworks',
    'dumping': 'Dumping',
    'trash': 'Trash',
    'garbage': 'Trash',
    'compost': 'Trash',
    'recycling': 'Trash',
    'rent': 'Housing',
    'rental': 'Housing',
    'rentals': 'Housing',
    'apartment': 'Housing',
    'apartments': 'Housing',
    'policy': 'Policy',
    'tax': 'Policy',
    'taxes': 'Policy',
    'mayor': 'Policy',
    'council': 'Policy',
    'councilwoman': 'Policy',
    'councilman': 'Policy',
    'environmental': 'Policy',
    'environment': 'Policy',
    'rezoning': 'Policy',
    'rezone': 'Policy',
    'government': 'Policy',
    'racing': 'Street Racing',
    'transit': 'Transit',
    'traffic': 'Transit',
    'pedestrian': 'Transit',
    'intersection': 'Transit',
    'bicycle': 'Transit',
    'bike': 'Transit',
    'speed': 'Transit',
    'pavement': 'Transit',
    'parks': 'Parks',
    'park': 'Parks',
    'playground': 'Parks',
    'trails': 'Parks',
    'pool': 'Parks',
    'gym': 'Parks',
    'medians': 'Parks',
}

# Function to check if any key in dict is in the string
def check_topics(value, topic_dict):
    topics_set = set()
    for key in topic_dict:
        words = value.split()
        if key in words:
            topics_set.add(topic_dict[key])
    
    if len(topics_set) > 0:
        return ','.join(topics_set)
    return 'Other'

# Apply the function and create a new column
d4_emails_df['topics'] = d4_emails_df['nouns_adjs_verbs'].apply(lambda x: check_topics(x, topic_dict))
d4_emails_df.head()

,name,affected_address,email_address,case_number,date,constituent_email_1,d4_response_1,d4_staff_member,constituent_email_2,d4_response_2,nouns_adjs_verbs,topics
0,Ron,6864 East Bucknell Place,ron@email.com,0,2024-08-05,The lack of police presence and code enforcement is sending a growing message that these violations are not important…and that reckless behavior is not of great concern. Second item: affordable de...,"Good morning Ron, \n\nThank you for reaching out, and I apologize for the delayed response. Council Pro Tem Romero Campbell maintains regular communication with DPD District 3, which serves Southe...",Staff1,NaN,NaN,sending concern affordable second item police great enforcement lack tax message wanting more denver violations behavior set important accomplish code goals presence reckless growing information,"Police,Policy"
1,Renee,Dahlia & High Line intersection,renee@email.com,0,2024-08-19,I would like to know if there is a possible to put a yellow flashing pedestrian light on Dahlia and the intersection of the Highline Canal.. Also is there aywya that a turn signal could be impleme...,Thank you for reaching out to our office. The area engineers have looked at this intersection where the High Line Canal crosses Dahlia multiple times at our request and have determined that a fla...,Staff2,NaN,NaN,put pedestrian light is aywya signal implemented intersection turn like flashing yellow possible know,Transit
2,JW,Happy Canyon & Jasmine St,jw@email.com,0,2024-08-19,"Accident took place at Happy Canyon & Jasmine. Last week an extremely careless driver drove at a high rate of speed and got stuck on the median island. Took out a tree, dug big holes in the ground...","Good morning TJ,\n\nThank you for reaching out to our office and for bringing this to our attention. I have reported this to the city via 311 and it will be directed to the parks and recreation te...",Staff1,NaN,NaN,place got removal speed road report various spilled driver fluids person stuff big stuck escape week holes island is dug city truck took inform drove oversaw last imagine high ground trying fixes ...,"Pothole,Transit"
3,Clara,Hutchinson Hills,clara@email.com,9578014,2024-08-20,"Has many concerns about getting a compost cart, had many issues with Solid Waste and the person they were talking to. Been trying to get a compost cart for 2 years since moving into their house. N...","Thank you for reaching out to our office and for your thoughtful email. I hope I can touch on all your points in my response.\nUnfortunately, it is not possible for you to pick up a compost bin. ...",Staff2,"Clara sent another email back with more questions about how to volunteer, wants a new rec center, and still wants more info on compost cart.",Did not respond as they were the same questions I had answewred previously,needs get compost many issues person house years has talking concerns message cart moving had getting trying,Trash
4,Pauline,NaN,pauline@email.com,0,2024-08-21,"At any rate my concern is how will HOA condo units be charges? $150.00 per unit per year so a 200 unit building would pay $40,000 additional on storm water bill. I look forward to clarification o...","Good afternoon Pauline, \n\nThank you for reaching out to our office. As you know the sidewalk fee in November 2022, a citizen-led ballot initiative passed by voters directed Denver's Department o...",Staff1,NaN,NaN,have general be unit units wage concern year budgets clarification inflation tax rules condo losing building lookup possible class made mentioned end scoop new middle regs folks bill tool fee pay ...,"Neighborhood,Policy"


In [295]:
# Drop the 'nouns_adjs_verbs' column
d4_topic_df = d4_emails_df.drop(columns=['nouns_adjs_verbs'])
# Save the dataframe to a csv file
d4_topic_df.to_csv('../resources/d4_emails_topics.csv', index=False)